# News Preprocessing - Load dữ liệu từ các file CSV

In [19]:
import json, os, ast, re, sys
import csv
import unicodedata as ud

from text_utils import remove_stopwords
from utils import load_n_gram, load_stopwords

In [20]:
data_path = './Data/'

csv_text_label_226 = 'vn_news_226_tlfr.csv'

In [25]:
def token_sylabling(text):
    
    text = ud.normalize('NFC', text)
    
    sign = ['==>', '=>', '->', '\.\.\.', '>>']
    digits = '\d+([\.,_]\d+)+'
    email = '(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)'
    web = '^(http[s]?://)?(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+$'
    datetime = [
        '\d{1,2}\/\d{1,2}(\/\d+)?',
        '\d{1,2}-\d{1,2}(-\d+)?',
    ]
    word = '\w+'
    non_word = '[^\w\s]'
    abbreviations = [
        '[A-ZĐ]+\.',
        'Tp\.',
        'Mr\.', 'Mrs\.', 'Ms\.',
        'Dr\.', 'ThS\.'
    ]
    
    patterns = []
    patterns.extend([web, email])
    patterns.extend(abbreviations)
    patterns.extend(sign)    
    patterns.extend(datetime)
    patterns.extend([digits, non_word, word])
    patterns = '(' + '|'.join(patterns) + ')'
    
    if sys.version_info < (3, 0):
            patterns = patterns.decode('utf-8')
    tokens = re.findall(patterns, text, re.UNICODE)
    return [token[0] for token in tokens]

## Thử nghiệm

In [28]:
text = "ngày 21/11/2018, Đây là game GARRY'S MOD ==> => chơi trên kênh Trực Tiếp Game của tôi - Dũng CT. tp.HCM, TP.HCM Rất mong nhận được sự ủng hộ của ae. ++ Đăng ký theo dõi kênh tại đây: https://goo.gl/A7BCZV ++ LINK TẢI APP CUBETV: https://www.cubetv.sg ++ THEO DÕI TTG TRÊN CUBETV: http://www.cubetv.sg/18947372 ++ DONATE ĐỂ TTG MUA ĐƯỢC NHIỀU GAME HƠN TẠI: - https://playerduo.com/tructiepgame - https://streamlabs.com/tructiepgamevn ++ MUA/THUÊ GAME BẢN QUYỀN GIÁ RẺ TẠI: Website: http://divineshop.vn Fanpage: https://www.facebook.com/Divine.Shop...."
token = token_sylabling(text)
print(token)

['ngày', '21/11/2018', ',', 'Đây', 'là', 'game', 'GARRY', "'", 'S', 'MOD', '==>', '=>', 'chơi', 'trên', 'kênh', 'Trực', 'Tiếp', 'Game', 'của', 'tôi', '-', 'Dũng', 'CT.', 'tp', '.', 'HCM', ',', 'TP.', 'HCM', 'Rất', 'mong', 'nhận', 'được', 'sự', 'ủng', 'hộ', 'của', 'ae', '.', '+', '+', 'Đăng', 'ký', 'theo', 'dõi', 'kênh', 'tại', 'đây', ':', 'https', ':', '/', '/', 'goo', '.', 'gl', '/', 'A7BCZV', '+', '+', 'LINK', 'TẢI', 'APP', 'CUBETV', ':', 'https', ':', '/', '/', 'www', '.', 'cubetv', '.', 'sg', '+', '+', 'THEO', 'DÕI', 'TTG', 'TRÊN', 'CUBETV', ':', 'http', ':', '/', '/', 'www', '.', 'cubetv', '.', 'sg', '/', '18947372', '+', '+', 'DONATE', 'ĐỂ', 'TTG', 'MUA', 'ĐƯỢC', 'NHIỀU', 'GAME', 'HƠN', 'TẠI', ':', '-', 'https', ':', '/', '/', 'playerduo', '.', 'com', '/', 'tructiepgame', '-', 'https', ':', '/', '/', 'streamlabs', '.', 'com', '/', 'tructiepgamevn', '+', '+', 'MUA', '/', 'THUÊ', 'GAME', 'BẢN', 'QUYỀN', 'GIÁ', 'RẺ', 'TẠI', ':', 'Website', ':', 'http', ':', '/', '/', 'divineshop', '